In [9]:
import re
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
from joblib import load
import nltk

# Убедитесь, что NLTK и его компоненты для русского языка установлены
# nltk.download('stopwords') # Раскомментируйте, если необходимо загрузить стоп-слова

from nltk.corpus import stopwords

# Загрузка моделей (замените пути на актуальные для ваших файлов)
model_lr = load('models/best_model_lr.joblib')
model_rf = load('models/best_model_rf.joblib')
model_xgb = load('models/best_model_xgb.joblib')

# Загрузка модели Word2Vec
word2vec_model = Word2Vec.load("word2vec.model")

# Определение стоп-слов
stop_words = set(stopwords.words('russian'))

# Инициализация стеммера
stemmer = SnowballStemmer('russian')

def clean_text(text):
    # Очистка и стемминг
    text = re.sub(r'\W', ' ', str(text))
    text = text.lower()
    text = re.sub(r'\s+[а-яА-Я]\s+', ' ', text)
    text = re.sub(r'\^[а-яА-Я]\s+', ' ', text) 
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub(r'^b\s+', '', text)
    return ' '.join([stemmer.stem(word) for word in text.split() if word not in stop_words])

def comment_to_avg_vector(comment, model):
    words = comment.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(word_vectors, axis=0)

def predict_class(text):
    prepared_text = clean_text(text)
    text_vector = comment_to_avg_vector(prepared_text, word2vec_model).reshape(1, -1) # Для одного примера, переформатируем в 2D массив
    predictions = {
        'Logistic Regression': model_lr.predict(text_vector)[0],
        'Random Forest': model_rf.predict(text_vector)[0],
        'XGBoost': model_xgb.predict(text_vector)[0]
    }
    return predictions



In [15]:
# Пример использования
text = ""
predictions = predict_class(text)
print(predictions)

{'Logistic Regression': 1.0, 'Random Forest': 0.0, 'XGBoost': 1}
